# Ryan Waterman

## Homework 4 SQL Questions

### Exercise 3-1

Retrieve the employee ID, first name, and last name for all bank employees. Sort by last name and then by first name.

In [24]:
!pip install dotenv psycopg2

In [25]:
import os
import psycopg2
import pandas as pd
from dotenv import find_dotenv, dotenv_values


In [26]:
keys = list(dotenv_values(find_dotenv('.env')).items())
os.environ['POSTGRES_PASS'] = keys[1][1]
print(os.getenv('POSTGRES_PASS'))

password


In [27]:
conn = psycopg2.connect(
        host="localhost",
        database="bank",
        user="Lab_03",
        password=os.getenv('POSTGRES_PASS'),
        port="5432"
    )

conn.autocommit=True

In [28]:
# Instantiate the cursor
cursor = conn.cursor()

In [29]:
#Execute the query
cursor.execute("SELECT emp_id, fname, lname FROM employee")

#Get the response
response = cursor.fetchall()

In [30]:
#Here is one approach to parsing the response into a df...
df = pd.DataFrame(
    {
        'emp_id':[item[0] for item in response],
        'fname':[item[1]for item in response],
        'lname':[item[2]for item in response]
    }
    )
df.head(5)

,emp_id,fname,lname
0,1,Michael,Smith
1,2,Susan,Barker
2,3,Robert,Tyler
3,4,Susan,Hawthorne
4,5,John,Gooding


In [ ]:
# I think a better way to do this is to read in the whole response and assign the columns in the df construction
cursor.execute("SELECT emp_id, fname, lname FROM employee")

df1 = pd.DataFrame(data=cursor.fetchall(), columns=['emp_id', 'fname', 'lname'])
df1.head(5)

,emp_id,fname,lname
0,1,Michael,Smith
1,2,Susan,Barker
2,3,Robert,Tyler
3,4,Susan,Hawthorne
4,5,John,Gooding


In [ ]:
#Anywho... let's finish the problem. This should sort by last, then first
df.sort_values(by='lname').sort_values(by='fname')

,emp_id,fname,lname
16,17,Beth,Fowler
6,7,Chris,Tucker
13,14,Cindy,Mason
14,15,Frank,Portman
5,6,Helen,Fleming
8,9,Jane,Grossman
12,13,John,Blake
4,5,John,Gooding
0,1,Michael,Smith
9,10,Paula,Roberts


As expected, the sort function can be chained together... Very convenient.

Now, try this with just a SQL query

In [42]:
cursor.execute("SELECT emp_id, fname, lname FROM employee ORDER BY fname ASC, lname ASC;")
cursor.fetchall()

[(17, 'Beth', 'Fowler'),
 (7, 'Chris', 'Tucker'),
 (14, 'Cindy', 'Mason'),
 (15, 'Frank', 'Portman'),
 (6, 'Helen', 'Fleming'),
 (9, 'Jane', 'Grossman'),
 (13, 'John', 'Blake'),
 (5, 'John', 'Gooding'),
 (1, 'Michael', 'Smith'),
 (10, 'Paula', 'Roberts'),
 (18, 'Rick', 'Tulman'),
 (3, 'Robert', 'Tyler'),
 (12, 'Samantha', 'Jameson'),
 (8, 'Sarah', 'Parker'),
 (2, 'Susan', 'Barker'),
 (4, 'Susan', 'Hawthorne'),
 (16, 'Theresa', 'Markham'),
 (11, 'Thomas', 'Ziegler')]

Okay, so this can be done within in the SQL query, I just needed to swap the sort order.

### Exercise 3-2
Retrieve the account ID, customer ID, and available balance for all accounts whose status equals 'ACTIVE' and whose available balance is greater than $2,500.

In [ ]:
#execute the query... this looks like its well suited for just a SQL command
cursor.execute("SELECT account_id, cust_id, avail_balance FROM account WHERE status = 'ACTIVE' and avail_balance > '2500';")
cursor.fetchall()


[(3, 1, 3000.0),
 (12, 4, 5487.09),
 (15, 6, 10000.0),
 (17, 7, 5000.0),
 (18, 8, 3487.19),
 (22, 9, 9345.55),
 (24, 10, 23575.12),
 (27, 11, 9345.55),
 (28, 12, 38552.05),
 (29, 13, 50000.0)]

### Exercise 3-3

Write a query against the account table that returns the IDs of the employees who opened the accounts (use the account.open_emp_id column). Include a single row for each distinct employee.

In [50]:
cursor.execute("SELECT DISTINCT open_emp_id FROM account")
cursor.fetchall()

[(13,), (10,), (1,), (16,)]

### Exercise 4-3

Construct a query that retrieves all accounts opened in 2002.